# Inspeksjon av Collection-datasettet

In [ ]:
%load_ext autoreload
%autoreload 2

figsize=(14, 4)
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from utils import load_collection_data, pairplot
pd.set_option('display.max_columns', None)
data_folder = os.path.join('../../..', 'data/prod')
file_name = "Collection_data.csv"
file_path = os.path.join(data_folder, file_name)

In [ ]:
df = load_collection_data(file_path)

### Datasettet
Datasettet består av kolonnene
- `yearmonth`: År og måned i formatet ÅÅÅÅMM
- `collectionid`: ID for inkassosaken
- `PersonId`: ID for person
- `registreringsmaaned`: registreringsperiode for når sak ble sendt til inkasso i formatet ÅÅÅÅMM
- `AccountId`: ID for konto
- `producttype`: Produkttype
- `productname`: Produktnavn
- `CollectionOpenedDateId`: Dato for når sak ble sendt til inkasso
- `CollectionClosedDateId`: Dato for når inkassosak ble lukket
- `BalanceSentAmt`: Balanse som ble sendt til inkasso (Hovedstol)
- `PaymentAmt`: Innbetalinger til Kredittbanken direkte  i perioden
- `PaymentRTAmt`: Sum innbetalt til Kredittbanken direkte på sak totalt 
- `PrincipalPaymentAmt`: Innbetalinger via inkassoselskapet i perioden
- `PrincipalPaymentRTAmt`: Sum innbetalinger gjort til inkassoselskapet totalt
- `LossAmt`: Tapsført i perioden
- `LossRTAmt`: Totalt tapsførst på sak
- `MonthInDCA`: Antall måneder sak har ligget hos inkasso
- `MonthsInZCOV`: Antall måneder sak har ligget til overvåk (tapsført)

In [ ]:
df["PersonId"].unique()

In [ ]:
pairplot(df, ['BalanceSentAmt', 'LossRTAmt', 'PrincipalPaymentRTAmt'])

In [ ]:
df.describe()

In [ ]:
df = df.sort_values(by=["PersonId", "yearmonth"], ascending=True)
df.head(30)

In [ ]:
#df['Status'] = np.where(df['LossRTAmt'].isna(), 'PENDING', 'COMPLETED')
df['Status'] = np.where(df['CollectionClosedDateId'].isna(), 'PENDING', 'COMPLETED')

In [ ]:
df = df.groupby(['PersonId', 'collectionid'], as_index=False).tail(1)
df.head(40)

In [ ]:
df['Loss'] = (df['LossRTAmt']/df['BalanceSentAmt'] * -1 * 100).round(2)
df[df['Status']=='COMPLETED'].head(40)

In [ ]:
figsize = (12,5)
fig, axes = plt.subplots(1, 2, figsize=figsize)

# First plot: 0 to 110
min1, max1 = 0, 110
bins1 = 25
loss_data1 = df['Loss'][(df['Loss'] >= min1) & (df['Loss'] <= max1)]

axes[0].hist(loss_data1, bins=bins1, range=(min1, max1), edgecolor='black')
axes[0].set_title(f'Distribution of Loss (%) from {min1}% to {max1}%')
axes[0].set_xlabel('Loss (%)')
axes[0].set_ylabel('Count')
axes[0].grid(axis='y', alpha=0.75)

# Second plot: 1 to 99
min2, max2 = 1, 99
bins2 = 25
loss_data2 = df['Loss'][(df['Loss'] >= min2) & (df['Loss'] <= max2)]

axes[1].hist(loss_data2, bins=bins2, range=(min2, max2), edgecolor='black')
axes[1].set_title(f'Distribution of Loss (%) from {min2}% to {max2}%')
axes[1].set_xlabel('Loss (%)')
axes[1].set_ylabel('Count')
axes[1].grid(axis='y', alpha=0.75)

plt.tight_layout()
plt.show()



In [ ]:
df[df["PersonId"] == 7216].head(20)

In [ ]:
col_dat = load_collection_data(file_path)

In [ ]:
col_dat.head(40)

In [ ]:
pd.set_option('display.max_rows', None)  # show all rows
print(col_dat.dtypes)
pd.reset_option('display.max_rows')

In [ ]:
col = "MonthsInZCOV"
col_dat[col].unique()

In [ ]:
col_dat[col].isna().any()

In [ ]:
col_dat["BalanceSentAmt"].max()